## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Honningsvag,NO,70.9821,25.9704,31.60,68,50,16.42,scattered clouds
1,1,Bathsheba,BB,13.2167,-59.5167,77.00,65,40,20.71,scattered clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,67.37,86,24,16.60,few clouds
3,3,Pevek,RU,69.7008,170.3133,-16.80,93,92,0.94,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,42.80,70,75,17.27,light snow


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 35
What is the maximum temperature you would like for your trip? 86


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bathsheba,BB,13.2167,-59.5167,77.00,65,40,20.71,scattered clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,67.37,86,24,16.60,few clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,42.80,70,75,17.27,light snow
5,5,Souillac,MU,-20.5167,57.5167,75.20,94,75,1.77,broken clouds
6,6,Katsuura,JP,35.1333,140.3000,69.01,34,20,10.36,few clouds
...,...,...,...,...,...,...,...,...,...,...
728,728,Northam,GB,51.0333,-4.2167,52.00,90,96,11.01,overcast clouds
730,730,Tupa,BR,-21.9347,-50.5136,72.91,68,9,6.38,clear sky
731,731,Esna,EG,25.2934,32.5540,55.40,47,0,6.91,clear sky
733,733,Margate,GB,51.3813,1.3862,50.00,80,57,1.99,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                544
City                   544
Country                544
Lat                    544
Lng                    544
Max Temp               544
Humidity               544
Cloudiness             544
Wind Speed             544
Current Description    544
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bathsheba,BB,77.00,scattered clouds,13.2167,-59.5167,
2,Port Alfred,ZA,67.37,few clouds,-33.5906,26.8910,
4,Ushuaia,AR,42.80,light snow,-54.8000,-68.3000,
5,Souillac,MU,75.20,broken clouds,-20.5167,57.5167,
6,Katsuura,JP,69.01,few clouds,35.1333,140.3000,
7,Turbat,PK,80.83,clear sky,26.0023,63.0440,
8,Nicoya,CR,78.80,clear sky,10.1483,-85.4520,
9,Avarua,CK,84.20,few clouds,-21.2078,-159.7750,
11,Esperance,AU,69.80,broken clouds,-33.8667,121.9000,
12,Cidreira,BR,73.51,light rain,-30.1811,-50.2056,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url=base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

SyntaxError: unexpected EOF while parsing (<ipython-input-13-174eaa224a18>, line 28)

In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].dropna()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bathsheba,BB,77.00,scattered clouds,13.2167,-59.5167,Atlantis Hotel
2,Port Alfred,ZA,67.37,few clouds,-33.5906,26.8910,The Halyards Hotel
4,Ushuaia,AR,42.80,light snow,-54.8000,-68.3000,Albatros Hotel
5,Souillac,MU,75.20,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
6,Katsuura,JP,69.01,few clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
...,...,...,...,...,...,...,...
728,Northam,GB,52.00,overcast clouds,51.0333,-4.2167,Durrant House Hotel
730,Tupa,BR,72.91,clear sky,-21.9347,-50.5136,Grande Hotel Tamoios
731,Esna,EG,55.40,clear sky,25.2934,32.5540,Orient Star Dahabyia Isna 05 Nights Each Sunday
733,Margate,GB,50.00,broken clouds,51.3813,1.3862,Premier Inn Margate hotel


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))